In [1]:
import numpy as np
import pandas as pd    

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
def relu(x):
    return np.maximum(0,x)

In [4]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [5]:
def initialize(x, neuron_num1, neuron_num2):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num1))-1
    w2=2*np.random.random((neuron_num1, neuron_num2))-1
    w3=2*np.random.random((neuron_num2, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    bias3=2*np.random.random()-1
    return w1, w2, w3, bias1, bias2, bias3

In [6]:
def forward_prop(w1, w2, w3, bias1, bias2, bias3, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    z3=np.dot(w3.T,out2)+bias3
    out3=relu(z3)
    return z1, out1, z2, out2, z3, out3 

In [7]:
def back_prop(w1, w2, w3, bias1, bias2, bias3, x, y, z1, out1, z2, out2, z3, out3, lr, j):
    x=x.reshape(13,1)
    
    err_3=2*(np.subtract(out3.sum(axis=0), y))
    dummy_3=np.array(deriv_relu(z3)).reshape(len(z3),1)
    dc_dw3=lr*np.dot(out2,dummy_3)*err_3
    w3=np.subtract(w3,dc_dw3)
    dc_db3=lr*err_3
    bias3-=dc_db3
    
    
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(len(z2),1)
    dc_dw2=lr*np.dot(out1,dummy_2.T)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(len(z1),1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, w3, bias1, bias2, bias3

In [8]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [9]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [10]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [11]:
target=ds.loc[:,'MEDV'].copy()
data=ds.drop('MEDV', axis=1).copy()

In [12]:
neuron_num1=7
neuron_num2=7

In [13]:
#data_normalized=(data.to_numpy()-min(data.to_numpy()))/(max(data.to_numpy())-min(data.to_numpy()))

In [14]:
w1, w2, w3, bias1, bias2, bias3 = initialize(data, neuron_num1, neuron_num2)

In [15]:
data_normalized = ((data - data.mean()) / data.std(ddof=0)).to_numpy()
target=target.to_numpy()

In [16]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [17]:
epochs=100
lr=0.0001

In [18]:
output1=list()
output2=list()
output3=list()
z_1=list()
z_2=list()
z_3=list()
loss=0
loss_test=0

In [19]:
for e in range(epochs):
    output1.clear()
    output2.clear()
    output3.clear()
    z_1.clear()
    z_2.clear()
    z_3.clear()
    errors=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2, z3, out3= forward_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i])
        output1.append(out1)
        output2.append(out2)
        output3.append(out3)
        z_1.append(z1)
        z_2.append(z2)
        z_3.append(z3)
       # predicted[i]=output[i].sum(axis=0)
        w1, w2, w3, bias1, bias2, bias3 = back_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i], y_train[i], z_1[i],
                                                    output1[i], z_2[i], output2[i], z_3[i], output3[i], lr, i+1)

In [20]:
print(w1.shape)

(13, 7)


In [21]:
print(w2.shape)

(7, 7)


In [22]:
print(w3.shape)

(7, 1)


In [23]:
for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
    loss+=(y_train[i]-(output3[i].sum(axis=0)))
print(loss/(len(y_train)))

[-0.08725909]


In [24]:
#for i in range(len(y_train)):
#    print(y_train[i], output3[i])

In [25]:
out_test=list()
for i in range(len(y_test)): 
    z_test1, out_test1, z_test2, out_test2, z_test3, out_test3=forward_prop(w1, w2, w3, bias1, bias2, bias3, x_test[i])
    out_test.append(out_test3)
    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
print(loss_test/(len(y_test)))

[-0.67854131]


In [26]:
for i in range(len(y_test)):
    print(y_test[i], out_test[i])

8.5 [[6.12801415]]
5.0 [[5.64605856]]
11.9 [[14.30623683]]
27.9 [[21.89054987]]
17.2 [[11.09049336]]
27.5 [[14.51694213]]
15.0 [[13.84544741]]
17.2 [[12.45622665]]
17.9 [[9.14685193]]
16.3 [[8.52921589]]
7.0 [[8.3866253]]
7.2 [[7.04358473]]
7.5 [[10.59816852]]
10.4 [[7.37656298]]
8.8 [[7.53897525]]
8.4 [[11.56737947]]
16.7 [[16.65644579]]
14.2 [[17.09517649]]
20.8 [[17.51917167]]
13.4 [[8.64085024]]
11.7 [[13.50627053]]
8.3 [[7.33936734]]
10.2 [[13.85476267]]
10.9 [[11.46164295]]
11.0 [[9.56696746]]
9.5 [[8.92944828]]
14.5 [[15.13750303]]
14.1 [[14.28471235]]
16.1 [[21.8358383]]
14.3 [[14.70034038]]
11.7 [[13.33800681]]
13.4 [[10.10228714]]
9.6 [[11.9678726]]
8.7 [[6.77241541]]
8.4 [[6.90833345]]
12.8 [[10.48427395]]
10.5 [[7.36803843]]
17.1 [[12.04477888]]
18.4 [[18.81969894]]
15.4 [[13.07117355]]
10.8 [[7.50620927]]
11.8 [[8.93362706]]
14.9 [[14.37058748]]
12.6 [[14.68302301]]
14.1 [[13.57984114]]
13.0 [[12.86699613]]
13.4 [[14.79029728]]
15.2 [[16.42601349]]
16.1 [[17.75030666]]
17.